In [96]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict
from surprise.model_selection import GridSearchCV
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import ast

In [97]:
movies = pd.read_csv('new_movies_missing_vals.csv')
ratings = pd.read_csv('../data/ratings.csv')
links = pd.read_csv('../data/links.csv',dtype=str)
ratings['rating'] = ratings['rating']*2
def convert(id):
    try:
        return str(int(id))
    except:
        return id
movies['tmdbId'] = movies['tmdbId'].apply(convert)
links = links.drop_duplicates(subset='tmdbId', keep='first')
movies = movies.merge(links[['tmdbId','imdbId']], how='inner', on='tmdbId')

In [98]:
movies[['vote_average','vote_count']].head()

,vote_average,vote_count
0,7.969,18889.0
1,7.237,10783.0
2,6.500,398.0
3,6.300,173.0
4,6.237,754.0


In [99]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movieId                9742 non-null   int64  
 1   tmdbId                 9734 non-null   object 
 2   adult                  9623 non-null   object 
 3   backdrop_path          9513 non-null   object 
 4   belongs_to_collection  2068 non-null   object 
 5   budget                 9623 non-null   float64
 6   genres                 9742 non-null   object 
 7   homepage               2639 non-null   object 
 8   id                     9623 non-null   float64
 9   imdb_id                9622 non-null   object 
 10  origin_country         9623 non-null   object 
 11  original_language      9623 non-null   object 
 12  original_title         9623 non-null   object 
 13  overview               9621 non-null   object 
 14  popularity             9623 non-null   float64
 15  post

In [100]:
missing = movies['vote_average'].isna()
missing

0       False
1       False
2       False
3       False
4       False
        ...  
9737    False
9738    False
9739    False
9740    False
9741    False
Name: vote_average, Length: 9742, dtype: bool

In [101]:
movies[missing][['vote_average','vote_count']]

,vote_average,vote_count
624,NaN,NaN
843,NaN,NaN
2141,NaN,NaN
3027,NaN,NaN
3127,NaN,NaN
...,...,...
9603,NaN,NaN
9634,NaN,NaN
9651,NaN,NaN
9690,NaN,NaN


In [102]:
movies[['vote_average','vote_count']].head()

,vote_average,vote_count
0,7.969,18889.0
1,7.237,10783.0
2,6.500,398.0
3,6.300,173.0
4,6.237,754.0


In [103]:
movies.isna().sum()

movieId                     0
tmdbId                      8
adult                     119
backdrop_path             229
belongs_to_collection    7674
budget                    119
genres                      0
homepage                 7103
id                        119
imdb_id                   120
origin_country            119
original_language         119
original_title            119
overview                  121
popularity                119
poster_path               124
production_companies      119
production_countries      119
release_date              119
revenue                   119
runtime                   119
spoken_languages          119
status                    119
tagline                  1440
title                       0
video                     119
vote_average              119
vote_count                119
imdbId                      0
dtype: int64

In [104]:
movies[['popularity','vote_count']].head(20)

,popularity,vote_count
0,21.4021,18889.0
1,3.0047,10783.0
2,1.4596,398.0
3,1.6120,173.0
4,2.0869,754.0
5,15.2539,7646.0
6,4.3758,644.0
7,1.0475,197.0
8,2.1873,746.0
9,7.8439,4025.0


In [105]:
# Step 1: Select relevant numeric columns that correlate with popularity
features = ['popularity', 'vote_average', 'vote_count', 'runtime', 'budget', 'revenue']

# Step 2: Create a working subset
df_median = movies[features].copy()

# Step 3: Fill budget and revenue with zero
df_median[['budget', 'revenue']] = df_median[['budget', 'revenue']].fillna(0)

# Step 4: Fill the rest with median
for col in ['popularity', 'vote_average', 'vote_count', 'runtime']:
    df_median[col] = df_median[col].fillna(df_median[col].median())

# Final imputed DataFrame
imputed_df = df_median


In [106]:
movies[features][missing].head(15)

,popularity,vote_average,vote_count,runtime,budget,revenue
624,NaN,NaN,NaN,NaN,NaN,NaN
843,NaN,NaN,NaN,NaN,NaN,NaN
2141,NaN,NaN,NaN,NaN,NaN,NaN
3027,NaN,NaN,NaN,NaN,NaN,NaN
3127,NaN,NaN,NaN,NaN,NaN,NaN
3362,NaN,NaN,NaN,NaN,NaN,NaN
3680,NaN,NaN,NaN,NaN,NaN,NaN
3741,NaN,NaN,NaN,NaN,NaN,NaN
4981,NaN,NaN,NaN,NaN,NaN,NaN
4986,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
for col in features:
    movies.loc[movies[col].isna(), col] = imputed_df.loc[movies[col].isna(), col]

In [108]:
movies[['imdbId']+features][missing].head(25)

,imdbId,popularity,vote_average,vote_count,runtime,budget,revenue
624,0113610,1.8512,6.569,471.0,102.0,0.0,0.0
843,0113610,1.8512,6.569,471.0,102.0,0.0,0.0
2141,0113610,1.8512,6.569,471.0,102.0,0.0,0.0
3027,0113610,1.8512,6.569,471.0,102.0,0.0,0.0
3127,0100232,1.8512,6.569,471.0,102.0,0.0,0.0
3362,0096913,1.8512,6.569,471.0,102.0,0.0,0.0
3680,0270933,1.8512,6.569,471.0,102.0,0.0,0.0
3741,0081809,1.8512,6.569,471.0,102.0,0.0,0.0
4981,0259153,1.8512,6.569,471.0,102.0,0.0,0.0
4986,0112130,1.8512,6.569,471.0,102.0,0.0,0.0


In [109]:
movies.isna().sum()

movieId                     0
tmdbId                      8
adult                     119
backdrop_path             229
belongs_to_collection    7674
budget                      0
genres                      0
homepage                 7103
id                        119
imdb_id                   120
origin_country            119
original_language         119
original_title            119
overview                  121
popularity                  0
poster_path               124
production_companies      119
production_countries      119
release_date              119
revenue                     0
runtime                     0
spoken_languages          119
status                    119
tagline                  1440
title                       0
video                     119
vote_average                0
vote_count                  0
imdbId                      0
dtype: int64

In [110]:
movies.to_csv('new_movies.csv', index=False)

In [111]:
cols_of_interset = ['movieId','genres', 'popularity', 'runtime', 'vote_average', 'vote_count']
movie_dataset = movies[cols_of_interset].copy()
movie_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieId       9742 non-null   int64  
 1   genres        9742 non-null   object 
 2   popularity    9742 non-null   float64
 3   runtime       9742 non-null   float64
 4   vote_average  9742 non-null   float64
 5   vote_count    9742 non-null   float64
dtypes: float64(4), int64(1), object(1)
memory usage: 456.8+ KB


In [112]:
movie_dataset.head()

,movieId,genres,popularity,runtime,vote_average,vote_count
0,1,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",21.4021,81.0,7.969,18889.0
1,2,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",3.0047,104.0,7.237,10783.0
2,3,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1.4596,101.0,6.500,398.0
3,4,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1.6120,127.0,6.300,173.0
4,5,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",2.0869,106.0,6.237,754.0


In [113]:
# Step 1: Safely convert string to list of dicts (if it's stored as a string)
def parse_genres(g):
    if isinstance(g, str):
        try:
            return ast.literal_eval(g)
        except:
            return []
    return g

movies['genres_parsed'] = movies['genres'].apply(parse_genres)

# Step 2: Extract genre names
all_genres = set()
for genre_list in movies['genres_parsed']:
    for genre in genre_list:
        if isinstance(genre, dict) and 'name' in genre:
            all_genres.add(genre['name'])

# Step 3: Get as sorted list
all_genres = sorted(all_genres)
all_genres


['(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'IMAX',
 'Music',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

In [114]:
all_genres = all_genres[1:]
all_genres

['Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'IMAX',
 'Music',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

In [115]:
for genre in all_genres:
    movie_dataset[genre] = movie_dataset['genres'].apply(lambda x: genre in x).astype(int)

In [116]:
movie_dataset.head()

,movieId,genres,popularity,runtime,vote_average,vote_count,Action,Adventure,Animation,Children,...,Music,Musical,Mystery,Romance,Sci-Fi,Science Fiction,TV Movie,Thriller,War,Western
0,1,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",21.4021,81.0,7.969,18889.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",3.0047,104.0,7.237,10783.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1.4596,101.0,6.500,398.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,4,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1.6120,127.0,6.300,173.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",2.0869,106.0,6.237,754.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
movie_dataset = movie_dataset.drop(columns='genres')

In [118]:
movie_dataset

,movieId,popularity,runtime,vote_average,vote_count,Action,Adventure,Animation,Children,Comedy,...,Music,Musical,Mystery,Romance,Sci-Fi,Science Fiction,TV Movie,Thriller,War,Western
0,1,21.4021,81.0,7.969,18889.0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,3.0047,104.0,7.237,10783.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1.4596,101.0,6.500,398.0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
3,4,1.6120,127.0,6.300,173.0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,5,2.0869,106.0,6.237,754.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,1.6639,100.0,7.600,93.0,1,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0
9738,193583,2.7855,106.0,7.800,408.0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
9739,193585,0.3624,96.0,6.806,18.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9740,193587,6.0790,90.0,8.123,163.0,1,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0


In [119]:
movie_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movieId          9742 non-null   int64  
 1   popularity       9742 non-null   float64
 2   runtime          9742 non-null   float64
 3   vote_average     9742 non-null   float64
 4   vote_count       9742 non-null   float64
 5   Action           9742 non-null   int32  
 6   Adventure        9742 non-null   int32  
 7   Animation        9742 non-null   int32  
 8   Children         9742 non-null   int32  
 9   Comedy           9742 non-null   int32  
 10  Crime            9742 non-null   int32  
 11  Documentary      9742 non-null   int32  
 12  Drama            9742 non-null   int32  
 13  Family           9742 non-null   int32  
 14  Fantasy          9742 non-null   int32  
 15  History          9742 non-null   int32  
 16  Horror           9742 non-null   int32  
 17  IMAX          

In [120]:
movie_dataset.isna().sum()

movieId            0
popularity         0
runtime            0
vote_average       0
vote_count         0
Action             0
Adventure          0
Animation          0
Children           0
Comedy             0
Crime              0
Documentary        0
Drama              0
Family             0
Fantasy            0
History            0
Horror             0
IMAX               0
Music              0
Musical            0
Mystery            0
Romance            0
Sci-Fi             0
Science Fiction    0
TV Movie           0
Thriller           0
War                0
Western            0
dtype: int64

In [121]:
movie_dataset.to_csv('movies_feature_engineered.csv', index=False)

In [122]:
movie_dataset2 = pd.read_csv('movies_feature_engineered.csv')

In [123]:
ids1 = movies['movieId'].unique()
ids2 = movie_dataset2['movieId'].unique()

In [124]:
1076 in ids2

True

In [125]:
(ids1==ids2).all()

True